# CSC-321: Data Mining and Machine Learning
# Irene Yin

## Assignment 8: Challenge Problem


### Part 1: Data manipulation
The test file is a distinct test file I want YOU to use to test your work. Only report scores using this test data. This means for THIS study you DO NOT do a cross validation. This is effectively a one holdout evaluation, where I GIVE you the train and test split. This is typical for competitions and evaluation events where we want to compare performance.

Remember, anything you do to the training data, you ALSO need to do to the test data.

I've also used the parameter skipinitialspace=True. If I DIDN'T, all my string values have an extra space at the beginning of them (so it wouldn't be "?", it would be " ?"), which is annoying. Pandas makes things easier, but data is dirty. Never be afraid to print out values so you can actually see them, and confirm WHAT they are exactly. DO NOT MAKE ASSUMPTIONS.

In [1]:
import pandas as pd

# Assign column names
labels = ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','wage']
missing_values = ["?"]

# Load the training data
# I've added column headings
# I've added the missing value string
# and in order to make that work, I've skipped the initial space

adult_train = pd.read_csv("https://raw.githubusercontent.com/nixwebb/CSV_Data/master/Adult/adult.data",names=labels,na_values = missing_values,skipinitialspace=True)

pre_rows,pre_cols = adult_train.shape
print("This is the Adult wage training data set. It has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")

# Show the head of the data
adult_train.head()

This is the Adult wage training data set. It has 32561 instances, and it has 15 features.




,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [2]:
# Load the test data
# Same loading as above, but I've skipped the first row

adult_test = pd.read_csv("https://raw.githubusercontent.com/nixwebb/CSV_Data/master/Adult/adult.test",names=labels,skiprows=1,na_values=missing_values,skipinitialspace=True)
pre_test_rows,pre_test_cols = adult_test.shape


# Show the head of the data
adult_test.head()

# deal with the wage data "."
adult_test['wage'] = adult_test['wage'].str.replace(r'.', '')
adult_test.head()



,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,NaN,103497,Some-college,10,Never-married,NaN,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# Let's try dropping unknowns - missing value "NaN"

print("\nDropping all instances with NaN values\n")

adult_train.dropna(inplace=True)
post_rows,post_cols = adult_train.shape
train_per = (pre_rows - post_rows) / pre_rows * 100

# I'm printing the total number of instances pre drop, post drop, and calculating the percentage dropped.

print("This is the Adult wage training data set.\nIt has",post_rows, "instances (down from",pre_rows,"), and it has", post_cols, "features (-",round(train_per,2),"%)\n\n")

adult_test.dropna(inplace=True)
post_test_rows,post_test_cols = adult_test.shape
test_per = (pre_test_rows - post_test_rows) / pre_test_rows * 100

print("This is the reduced Adult wage test data set.\nIt has", post_test_rows, "instances (down from",pre_test_rows,"), and it has", post_test_cols, "features (-",round(test_per,2),"%)\n\n")


Dropping all instances with NaN values

This is the Adult wage training data set.
It has 30162 instances (down from 32561 ), and it has 15 features (- 7.37 %)


This is the reduced Adult wage test data set.
It has 15060 instances (down from 16281 ), and it has 15 features (- 7.5 %)




We need to transform them TO numeric.

We could just replace the values with numbers in the columns, but remember that can lead to undesirable behavior. If one job category is assigned a 1, and another is a 3, is that second category REALLY 3 times better than the first?

You have to ultimately decide which encoding makes SENSE for your data, and that will NOT necessarily align with the highest performance. I want you to concentrate on EXPLAINABILITY.

Let's take a look at the feature value data types.

In [4]:
adult_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   fnlwgt          30162 non-null  int64 
 3   education       30162 non-null  object
 4   education-num   30162 non-null  int64 
 5   marital-status  30162 non-null  object
 6   occupation      30162 non-null  object
 7   relationship    30162 non-null  object
 8   race            30162 non-null  object
 9   sex             30162 non-null  object
 10  capital-gain    30162 non-null  int64 
 11  capital-loss    30162 non-null  int64 
 12  hours-per-week  30162 non-null  int64 
 13  native-country  30162 non-null  object
 14  wage            30162 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [5]:
adult_train.describe(include='all')

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
count,30162.000000,30162,3.016200e+04,30162,30162.000000,30162,30162,30162,30162,30162,30162.000000,30162.000000,30162.000000,30162,30162
unique,NaN,7,NaN,16,NaN,7,14,6,5,2,NaN,NaN,NaN,41,2
top,NaN,Private,NaN,HS-grad,NaN,Married-civ-spouse,Prof-specialty,Husband,White,Male,NaN,NaN,NaN,United-States,<=50K
freq,NaN,22286,NaN,9840,NaN,14065,4038,12463,25933,20380,NaN,NaN,NaN,27504,22654
mean,38.437902,NaN,1.897938e+05,NaN,10.121312,NaN,NaN,NaN,NaN,NaN,1092.007858,88.372489,40.931238,NaN,NaN
std,13.134665,NaN,1.056530e+05,NaN,2.549995,NaN,NaN,NaN,NaN,NaN,7406.346497,404.298370,11.979984,NaN,NaN
min,17.000000,NaN,1.376900e+04,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,1.000000,NaN,NaN
25%,28.000000,NaN,1.176272e+05,NaN,9.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
50%,37.000000,NaN,1.784250e+05,NaN,10.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,40.000000,NaN,NaN
75%,47.000000,NaN,2.376285e+05,NaN,13.000000,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,45.000000,NaN,NaN


Both sex and wage columns have only two features. We can safely encode those as numbers, because any comparison is basically a binary choice. In fact wage we MUST define as an integer, because it's our class.

Below, I'm using the LabelEncoder from sklearn to do this, first to the training data, and second to the test data. The nice thing with these encoders is that I can then apply them to any data, to transform it into the right thing, using the .transform method.

In [6]:
from sklearn import preprocessing

# Encoding sex in the training and test data --> Female: 0, Male: 1

le_sex = preprocessing.LabelEncoder()

vals = adult_train['sex'].values
le_sex.fit(vals)
adult_train['sex'] = le_sex.transform(vals)
adult_test['sex'] = le_sex.transform(adult_test['sex'])
adult_train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,<=50K


In [7]:
# Encoding wage in the training and test data. --> <=50: 0, >50k: 1

le_wage = preprocessing.LabelEncoder()
wage_vals = adult_train['wage'].values
le_wage.fit(wage_vals)
adult_train['wage'] = le_wage.transform(wage_vals)
adult_test['wage'] = le_wage.transform(adult_test['wage'].values)
adult_test.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,wage
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,1,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,1,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,1,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,1,7688,0,40,United-States,1
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,1,0,0,30,United-States,0


To dropp the instances that are AREN'T from the USA, first I find the index of where all the NOT UNITED STATES entries are, then I drop values where that index is true. I do this INPLACE - meaning I fundamentally change the dataframe. If I want to get those other countries back, I'll have to reload the dataframe.

THINK about memory usage here - do you NEED a new dataframe, or can you alter the existing one. Pandas provides this flexibility with a host of it's methods - look for methods that allow the INPLACE argument.

REMEMBER THIS: For your study (as with your final project), I must be able to run the notebook once, with no alterations, and NO ERRORS to get all the values, figures and graphs you want to show me.

Notice that I really do always print how many instances I now have.

In [8]:
# drop the instances which the value of the country is not USA.

#Get names of indexes for which column Age has value 30

indexCountry = adult_train[ adult_train['native-country'] != 'United-States' ].index
 
# Delete these row indexes from dataFrame
adult_train.drop(indexCountry , inplace=True)

usa_rows,usa_cols = adult_train.shape
usa_per = (post_rows - usa_rows) / post_rows * 100

print("This is the Adult wage training data set.\nIt has",usa_rows, "instances (down from",post_rows,"), and it has", usa_cols, "features (-",round(usa_per,2),"%)\n\n")

# Repeat to test data
indexCountry = adult_test[ adult_test['native-country'] != 'United-States' ].index
adult_test.drop(indexCountry , inplace=True)

adult_train_wage = adult_train['wage'].copy()
adult_test_wage = adult_test['wage'].copy()

adult_train.drop(['wage'],axis=1,inplace=True)
adult_test.drop(['wage'],axis=1,inplace=True)


This is the Adult wage training data set.
It has 27504 instances (down from 30162 ), and it has 15 features (- 8.81 %)




Now I've dropped all but the United States entries, let's go ahead and do one-hot encoding (or adding dummy variables, as it's also called).

In [9]:
# Get dummies
adult_train = pd.get_dummies(adult_train, prefix_sep='_', drop_first=True)
adult_test = pd.get_dummies(adult_test, prefix_sep='_', drop_first=True)

# X head
adult_train.head()

,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White
0,39,77516,13,1,2174,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
1,50,83311,13,1,0,0,13,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,38,215646,9,1,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,53,234721,7,1,0,0,40,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
5,37,284582,14,0,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1


In [10]:
# Adding the wage class column back in to the data

adult_train['wage']=adult_train_wage
adult_test['wage']=adult_test_wage

final_rows,final_cols = adult_train.shape

print("This is the Adult wage training data set.\nIt has",final_rows, "instances, and it has", final_cols, "features\n\n")

adult_train.head()

This is the Adult wage training data set.
It has 27504 instances, and it has 57 features




,age,fnlwgt,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,education_11th,education_12th,education_1st-4th,education_5th-6th,education_7th-8th,education_9th,education_Assoc-acdm,education_Assoc-voc,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,relationship_Unmarried,relationship_Wife,race_Asian-Pac-Islander,race_Black,race_Other,race_White,wage
0,39,77516,13,1,2174,0,40,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
1,50,83311,13,1,0,0,13,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,38,215646,9,1,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
3,53,234721,7,1,0,0,40,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,37,284582,14,0,0,0,40,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0


Now the data are all in numeric version.

In [11]:
pre_rows,pre_cols = adult_train.shape
print("This is the Adult wage training data set. It has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")

This is the Adult wage training data set. It has 27504 instances, and it has 57 features.




### Part 2 Selecting features

In this dataset, we don't need to use the Cross-validation because there are train and test dataset ready for us. 

We need to split the training data into X_train, y_train, and split the testing data into X_test, and y_test. 

For this particular data, we also need to do the normalization to the test and train dataset. We need to get the coefficient for each features to find the important features. (since 57 features are too much for the task where it might cause the overfitting error.)

This is a classfication task. According to the illustrations above, there is a linear pattern in the images. So we might want to use the linear method instead of non-linear method.  

For this classification task, we can consider the following methods: Naive Bayes, and Logistic Regression. We can also use KNN , the nonlinear method, to see how it performs, compare to the linear method


Now we need to do some feature selections based on our understadnings. Just seeing the attributes we already have, we can notice that there are few features that are duplicated with each other. Thus, we need to drop those duplicated features in order to make our model smaller. 

1. The education-num could represent the education level. If you are doctorate, then you definitely have more education-num compare to the HS-grad. Hence, we want to drop all the features start with eduction except education-num since the education-num could represent them.

2. The fnlwgt means the final weight: this is the number of people the census believes the entry represents. This does not relate to the wages of one could get. So we also want to drop this feature. 

3. The relationship features can be represetned by sex and marital status. Hence, we could also drop realtionship features. 


In [12]:
# drop eduction features
adult_train_copy = adult_train.copy() # create a copy of the original dataset.
adult_train_copy.drop('education_Some-college',axis=1,inplace=True)
adult_train_copy.drop('education_Prof-school',axis=1,inplace=True)
adult_train_copy.drop('education_Preschool',axis=1,inplace=True)
adult_train_copy.drop('education_Masters',axis=1,inplace=True)
adult_train_copy.drop('education_HS-grad',axis=1,inplace=True)
adult_train_copy.drop('education_Bachelors',axis=1,inplace=True)
adult_train_copy.drop('education_Assoc-voc',axis=1,inplace=True)
adult_train_copy.drop('education_Assoc-acdm',axis=1,inplace=True)
adult_train_copy.drop('education_9th',axis=1,inplace=True)
adult_train_copy.drop('education_7th-8th',axis=1,inplace=True)
adult_train_copy.drop('education_5th-6th',axis=1,inplace=True)
adult_train_copy.drop('education_1st-4th',axis=1,inplace=True)
adult_train_copy.drop('education_12th',axis=1,inplace=True)
adult_train_copy.drop('education_11th',axis=1,inplace=True)
adult_train_copy.drop('education_Doctorate',axis=1,inplace=True)

# drop fnlwgt
adult_train_copy.drop('fnlwgt',axis=1,inplace=True)

# drop relationship
adult_train_copy.drop('relationship_Wife',axis=1,inplace=True)
adult_train_copy.drop('relationship_Not-in-family',axis=1,inplace=True)
adult_train_copy.drop('relationship_Unmarried',axis=1,inplace=True)
adult_train_copy.drop('relationship_Own-child',axis=1,inplace=True)
adult_train_copy.drop('relationship_Other-relative',axis=1,inplace=True)

# combine features into one representative feature
adult_train_copy['workclass_Government'] = adult_train_copy['workclass_Local-gov'] + adult_train_copy['workclass_State-gov']
adult_train_copy.drop('workclass_Local-gov',axis=1,inplace=True)
adult_train_copy.drop('workclass_State-gov',axis=1,inplace=True)

adult_train_copy = adult_train_copy[['age', 'education-num',	'sex',	'capital-gain',	'capital-loss',	'hours-per-week','workclass_Private',	'workclass_Government',
                                   'workclass_Self-emp-inc',	'workclass_Self-emp-not-inc',	'workclass_Without-pay',
                                  'marital-status_Married-AF-spouse',	'marital-status_Married-civ-spouse',	'marital-status_Married-spouse-absent',
                                  'marital-status_Never-married',	'marital-status_Separated',	'marital-status_Widowed',	'occupation_Armed-Forces',
                                  'occupation_Craft-repair',	'occupation_Exec-managerial',	'occupation_Farming-fishing',	'occupation_Handlers-cleaners',
                                  'occupation_Machine-op-inspct',	'occupation_Other-service',	'occupation_Priv-house-serv',	'occupation_Prof-specialty',
                                  'occupation_Protective-serv',	'occupation_Sales',	'occupation_Tech-support',	'occupation_Transport-moving',
                                  'race_Asian-Pac-Islander',	'race_Black',	'race_Other',	'race_White',	'wage']]

pre_rows,pre_cols = adult_train_copy.shape
print("After dropping all the features we don't want. Here is the Adult wage training data set.\n"
        "It now has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")
adult_train_copy.head()



After dropping all the features we don't want. Here is the Adult wage training data set.
It now has 27504 instances, and it has 35 features.




,age,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Government,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,race_Asian-Pac-Islander,race_Black,race_Other,race_White,wage
0,39,13,1,2174,0,40,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,50,13,1,0,0,13,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,38,9,1,0,0,40,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
3,53,7,1,0,0,40,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0
5,37,14,0,0,0,40,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [13]:
# do the same thing to the test dataset.

# drop eduction features
adult_test_copy = adult_test.copy() # create a copy of the original dataset.
adult_test_copy.drop('education_Some-college',axis=1,inplace=True)
adult_test_copy.drop('education_Prof-school',axis=1,inplace=True)
adult_test_copy.drop('education_Preschool',axis=1,inplace=True)
adult_test_copy.drop('education_Masters',axis=1,inplace=True)
adult_test_copy.drop('education_HS-grad',axis=1,inplace=True)
adult_test_copy.drop('education_Bachelors',axis=1,inplace=True)
adult_test_copy.drop('education_Assoc-voc',axis=1,inplace=True)
adult_test_copy.drop('education_Assoc-acdm',axis=1,inplace=True)
adult_test_copy.drop('education_9th',axis=1,inplace=True)
adult_test_copy.drop('education_7th-8th',axis=1,inplace=True)
adult_test_copy.drop('education_5th-6th',axis=1,inplace=True)
adult_test_copy.drop('education_1st-4th',axis=1,inplace=True)
adult_test_copy.drop('education_12th',axis=1,inplace=True)
adult_test_copy.drop('education_11th',axis=1,inplace=True)
adult_test_copy.drop('education_Doctorate',axis=1,inplace=True)

# drop fnlwgt
adult_test_copy.drop('fnlwgt',axis=1,inplace=True)

# drop relationship
adult_test_copy.drop('relationship_Wife',axis=1,inplace=True)
adult_test_copy.drop('relationship_Not-in-family',axis=1,inplace=True)
adult_test_copy.drop('relationship_Unmarried',axis=1,inplace=True)
adult_test_copy.drop('relationship_Own-child',axis=1,inplace=True)
adult_test_copy.drop('relationship_Other-relative',axis=1,inplace=True)

# combine features into one representative feature
adult_test_copy['workclass_Government'] = adult_test_copy['workclass_Local-gov'] + adult_test_copy['workclass_State-gov']
adult_test_copy.drop('workclass_Local-gov',axis=1,inplace=True)
adult_test_copy.drop('workclass_State-gov',axis=1,inplace=True)

adult_test_copy = adult_test_copy[['age', 'education-num',	'sex',	'capital-gain',	'capital-loss',	'hours-per-week','workclass_Private',	'workclass_Government',
                                   'workclass_Self-emp-inc',	'workclass_Self-emp-not-inc',	'workclass_Without-pay',
                                  'marital-status_Married-AF-spouse',	'marital-status_Married-civ-spouse',	'marital-status_Married-spouse-absent',
                                  'marital-status_Never-married',	'marital-status_Separated',	'marital-status_Widowed',	'occupation_Armed-Forces',
                                  'occupation_Craft-repair',	'occupation_Exec-managerial',	'occupation_Farming-fishing',	'occupation_Handlers-cleaners',
                                  'occupation_Machine-op-inspct',	'occupation_Other-service',	'occupation_Priv-house-serv',	'occupation_Prof-specialty',
                                  'occupation_Protective-serv',	'occupation_Sales',	'occupation_Tech-support',	'occupation_Transport-moving',
                                  'race_Asian-Pac-Islander',	'race_Black',	'race_Other',	'race_White',	'wage']]
pre_rows,pre_cols = adult_test_copy.shape
print("After dropping all the features we don't want. Here is the Adult wage testing data set.\n"
        "It now has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")
adult_test_copy.head()



After dropping all the features we don't want. Here is the Adult wage testing data set.
It now has 13788 instances, and it has 35 features.




,age,education-num,sex,capital-gain,capital-loss,hours-per-week,workclass_Private,workclass_Government,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,marital-status_Married-spouse-absent,marital-status_Never-married,marital-status_Separated,marital-status_Widowed,occupation_Armed-Forces,occupation_Craft-repair,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving,race_Asian-Pac-Islander,race_Black,race_Other,race_White,wage
0,25,7,1,0,0,40,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
1,38,9,1,0,0,50,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,28,12,1,0,0,40,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1
3,44,10,1,7688,0,40,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1
5,34,6,1,0,0,30,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0


First, we need normalize both the training data and the testing data. We will split the training data into X_values and y_values. We will also split the testing data into X_values_test and y_values_test.

In [14]:
from sklearn.preprocessing import MinMaxScaler

# Extract the values from the pandas dataframe
adult_value = adult_train_copy.values

X_values = adult_value[:,:-1]
y_values = adult_value[:,len(adult_value[0])-1]

# Get the shape, and print meaningful things
# Make sure this agrees with the decribe method, above

rows,cols = X_values.shape
print("This is the wine data set. It has", rows, "instances, and it has", cols, "input features.\n")
print("The first FIVE instances look like:")

# Show the first five instances
print(X_values[:5])
print()

# Load and fit the scaler

scaler = MinMaxScaler()
scaler.fit(X_values)

# Use some attributes of the scaler to show min and max values per feature
# Note these should align with the information from the pandas .describe
# method, used above

print("MAX values:",scaler.data_max_)
print("MIN values:",scaler.data_min_)
print()

# Transform our X_values, so that data is now scaled
# Note we can apply this transform to any data, including new data
# and it will preserve the min and max values given above

X_values = scaler.transform(X_values)

# Take another look at those first five instances that should now be 
# normalized

print("After normalization, the first FIVE instances look like:")
print(X_values[:5])


This is the wine data set. It has 27504 instances, and it has 34 input features.

The first FIVE instances look like:
[[  39   13    1 2174    0   40    0    1    0    0    0    0    0    0
     1    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    1]
 [  50   13    1    0    0   13    0    0    0    1    0    0    1    0
     0    0    0    0    0    1    0    0    0    0    0    0    0    0
     0    0    0    0    0    1]
 [  38    9    1    0    0   40    1    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    1    0    0    0    0    0    0
     0    0    0    0    0    1]
 [  53    7    1    0    0   40    1    0    0    0    0    0    1    0
     0    0    0    0    0    0    0    1    0    0    0    0    0    0
     0    0    0    1    0    0]
 [  37   14    0    0    0   40    1    0    0    0    0    0    1    0
     0    0    0    0    0    1    0    0    0    0    0    0    0    0
     0    0    0    0    0    

In [15]:
# Normalization on Testing data


from sklearn.preprocessing import MinMaxScaler

# Extract the values from the pandas dataframe


adult_Value = adult_test_copy.values

X_values_test = adult_Value[:,:-1]
y_values_test = adult_Value[:,len(adult_Value[0])-1]

# Get the shape, and print meaningful things
# Make sure this agrees with the decribe method, above

rows,cols = X_values_test.shape
print("This is the wine data set. It has", rows, "instances, and it has", cols, "input features.\n")
print("The first FIVE instances look like:")

# Show the first five instances
print(X_values_test[:5])
print()

# Load and fit the scaler

scaler = MinMaxScaler()
scaler.fit(X_values_test)

# Use some attributes of the scaler to show min and max values per feature
# Note these should align with the information from the pandas .describe
# method, used above

print("MAX values:",scaler.data_max_)
print("MIN values:",scaler.data_min_)
print()

# Transform our X_values_test, so that data is now scaled
# Note we can apply this transform to any data, including new data
# and it will preserve the min and max values given above

X_values_test = scaler.transform(X_values_test)

# Take another look at those first five instances that should now be 
# normalized

print("After normalization, the first FIVE instances look like:")
print(X_values_test[:5])


This is the wine data set. It has 13788 instances, and it has 34 input features.

The first FIVE instances look like:
[[  25    7    1    0    0   40    1    0    0    0    0    0    0    0
     1    0    0    0    0    0    0    0    1    0    0    0    0    0
     0    0    0    1    0    0]
 [  38    9    1    0    0   50    1    0    0    0    0    0    1    0
     0    0    0    0    0    0    1    0    0    0    0    0    0    0
     0    0    0    0    0    1]
 [  28   12    1    0    0   40    0    1    0    0    0    0    1    0
     0    0    0    0    0    0    0    0    0    0    0    0    1    0
     0    0    0    0    0    1]
 [  44   10    1 7688    0   40    1    0    0    0    0    0    1    0
     0    0    0    0    0    0    0    0    1    0    0    0    0    0
     0    0    0    1    0    0]
 [  34    6    1    0    0   30    1    0    0    0    0    0    0    0
     1    0    0    0    0    0    0    0    0    1    0    0    0    0
     0    0    0    0    0    

I'm using the wrapper approach to do the feature selection.

After we get the coefficient for each feature, we will choose the features with the absolute coefficient value that is bigger than 0.6 as our training features.

In [29]:
# Feature Selection
import math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np
import math
# Add you code for (l) through (o) here
X_train = X_values
y_train = y_values


def pred(instance, coefficient):
  #return: a prediction
  exponent = coefficient[0]
  for i in range(len(coefficient)-1):
    exponent += coefficient[i+1]*instance[i]
  predY = 1.0/(1.0 + math.exp(-exponent))
  return predY


def sgd_log(X_train, y_train, learning_rate, epochs):
  # return the coefficients.
  coefficient = [0.0 for j in range(len(X_train[0])+1)]
  for iteration in range(epochs):
    Totalerror = 0
    for i in range(len(X_train)):
      preditedY = pred(X_train[i],coefficient)
      error = y_train[i] - preditedY
      Totalerror += error**2
      coefficient[0] += learning_rate * error * preditedY*(1.0-preditedY)
      for index in range(len(coefficient)-1):
        coefficient[index+1] += learning_rate * error *preditedY*(1.0-preditedY)* X_train[i][index]
  return coefficient
learning_rate = 0.01
epochs = 100

Coefficient = sgd_log(X_train, y_train, learning_rate, epochs)
print("the coefficient is: ", Coefficient)

print('INTERCEPT: (B0)',Coefficient[0])
print('COEFFS: (B1...BN)',Coefficient[1:])         

coefficient = Coefficient[1:]

fetures_name = ['age','education-num',	'sex',	'capital-gain',	'capital-loss',	'hours-per-week','workclass_Private',	'workclass_Government','workclass_Self-emp-inc',	'workclass_Self-emp-not-incis'
                ,'workclass_Without-pay','marital-status_Married-AF-spouse',	'marital-status_Married-civ-spouse',	'marital-status_Married-spouse-absent',	'marital-status_Never-married',	'marital-status_Separated',	
                'marital-status_Widowed',
                'occupation_Armed-Forces',	'occupation_Craft-repair',	'occupation_Exec-managerial',	'occupation_Farming-fishing',	'occupation_Handlers-cleaners',	'occupation_Machine-op-inspct',	'occupation_Other-service',
                'occupation_Priv-house-serv',	'occupation_Prof-specialty',	'occupation_Protective-serv',	'occupation_Sales',	'occupation_Tech-support',	'occupation_Transport-moving',
                'race_Asian-Pac-Islander',	'race_Black',	'race_Other',	'race_White']

# choose the features with the absolute value bigger than 0.6
def featureSelection(coefList, rangeNum, featureName):
    # return a list of the feature corresponding to its coefficient above the rangeNum
    selector = []
    for i in range(len(coefList)):
      i_abs = abs(coefList[i])
      if i_abs > rangeNum:
        selector.append((coefList[i],featureName[i]))
    return sorted(selector)

selected_features = featureSelection(coefficient, 0.6, fetures_name)

print()
print("The choosen features with the absolute coefficient value that is bigger than 0.6 are:\n\n")
for feature in selected_features:
  data = '%.3f'%feature[0]
  print('{:^40}'.format(feature[1]), ":", '%10s'%data)


the coefficient is:  [-6.1946587400318505, 1.7600979209049048, 4.181071460643242, 0.08516813452299726, 9.22902309999404, 2.252069509903865, 2.4145507672839996, -0.4458575094326102, -0.7175655981833132, -0.25820407196470624, -1.0182173317442373, -0.5663941893694265, 0.7885010081083501, 1.9426084573976534, -0.12256866671589702, -0.3251400870071444, 0.019414831850267367, -0.06686311040127739, -0.20290672561722398, -0.1058188537951349, 0.6344130556411235, -1.0144331587253994, -0.8956468397156768, -0.373466796214923, -1.001308279475937, -0.5531544633223019, 0.36463934620959937, 0.4028627922957338, 0.1585501023832149, 0.4501692585097336, -0.26527240429298254, 0.150945661087575, -0.03785098369576173, -0.5271699194170206, 0.0012597042772526511]
INTERCEPT: (B0) -6.1946587400318505
COEFFS: (B1...BN) [1.7600979209049048, 4.181071460643242, 0.08516813452299726, 9.22902309999404, 2.252069509903865, 2.4145507672839996, -0.4458575094326102, -0.7175655981833132, -0.25820407196470624, -1.01821733174423

The absolute value of the coefficient could help us to determine which feature values more. The bigger the absolute value of the coefficient, the more influential that corresponding feature is.  Based on calculating the Coefficient of each features, we choose the absolute coefficient values that are greater than 0.6. We then have 13 features choosen. See the listed features and values above.


In [18]:
# leave the 16 features we chose.
adult_train_06 = adult_train_copy.copy()
pre_rows,pre_cols = adult_train_06.shape
print("This is the Adult wage training data set. It has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")

adult_train_06.drop('race_Other',axis=1,inplace=True)
adult_train_06.drop('race_White',axis=1,inplace=True)
adult_train_06.drop('race_Black',axis=1,inplace=True)

adult_train_06.drop('occupation_Transport-moving',axis=1,inplace=True)
adult_train_06.drop('occupation_Tech-support',axis=1,inplace=True)
adult_train_06.drop('occupation_Sales',axis=1,inplace=True)
adult_train_06.drop('occupation_Prof-specialty',axis=1,inplace=True)
adult_train_06.drop('occupation_Machine-op-inspct',axis=1,inplace=True)
adult_train_06.drop('occupation_Protective-serv',axis=1,inplace=True)
adult_train_06.drop('occupation_Craft-repair',axis=1,inplace=True)
adult_train_06.drop('occupation_Armed-Forces',axis=1,inplace=True)

adult_train_06.drop('marital-status_Separated',axis=1,inplace=True)
adult_train_06.drop('marital-status_Never-married',axis=1,inplace=True)
adult_train_06.drop('marital-status_Married-spouse-absent',axis=1,inplace=True)
adult_train_06.drop('marital-status_Widowed',axis=1,inplace=True)

adult_train_06.drop('workclass_Self-emp-inc',axis=1,inplace=True)
adult_train_06.drop('workclass_Private',axis=1,inplace=True)
adult_train_06.drop('sex',axis=1,inplace=True)


#['INTERCEPT','age','fnlwgt','education-num',	'sex',	'capital-gain',	'capital-loss',	'hours-per-week',
# 'workclass_Without-pay',	'education_Doctorate', 'marital-status_Married-AF-spouse',	relationship_Not-in-family']

pre_rows,pre_cols = adult_train_06.shape
print("After dropping all the features we don't want. Here is the Adult wage training data set.\n"
        "It now has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")
adult_train_06.head()


This is the Adult wage training data set. It has 27504 instances, and it has 35 features.


After dropping all the features we don't want. Here is the Adult wage training data set.
It now has 27504 instances, and it has 17 features.




,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Government,workclass_Self-emp-not-inc,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Other-service,occupation_Priv-house-serv,race_Asian-Pac-Islander,wage
0,39,13,2174,0,40,1,0,0,0,0,0,0,0,0,0,0,0
1,50,13,0,0,13,0,1,0,0,1,1,0,0,0,0,0,0
2,38,9,0,0,40,0,0,0,0,0,0,0,1,0,0,0,0
3,53,7,0,0,40,0,0,0,0,1,0,0,1,0,0,0,0
5,37,14,0,0,40,0,0,0,0,1,1,0,0,0,0,0,0


Now we have only 10 features left in the training set data. We then need to also do the same thing to the testing set data. 

After we finished altering our data, we need to get to our classification task.

In [19]:
# leave the 16 features we chose.
adult_test_16 = adult_test_copy.copy()
pre_rows,pre_cols = adult_test_16.shape
print("This is the Adult wage training data set. It has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")

adult_test_16.drop('race_Other',axis=1,inplace=True)
adult_test_16.drop('race_White',axis=1,inplace=True)
adult_test_16.drop('race_Black',axis=1,inplace=True)

adult_test_16.drop('occupation_Transport-moving',axis=1,inplace=True)
adult_test_16.drop('occupation_Tech-support',axis=1,inplace=True)
adult_test_16.drop('occupation_Sales',axis=1,inplace=True)
adult_test_16.drop('occupation_Prof-specialty',axis=1,inplace=True)
adult_test_16.drop('occupation_Machine-op-inspct',axis=1,inplace=True)
adult_test_16.drop('occupation_Protective-serv',axis=1,inplace=True)
adult_test_16.drop('occupation_Craft-repair',axis=1,inplace=True)
adult_test_16.drop('occupation_Armed-Forces',axis=1,inplace=True)

adult_test_16.drop('marital-status_Separated',axis=1,inplace=True)
adult_test_16.drop('marital-status_Never-married',axis=1,inplace=True)
adult_test_16.drop('marital-status_Married-spouse-absent',axis=1,inplace=True)
adult_test_16.drop('marital-status_Widowed',axis=1,inplace=True)

adult_test_16.drop('workclass_Self-emp-inc',axis=1,inplace=True)
adult_test_16.drop('workclass_Private',axis=1,inplace=True)
adult_test_16.drop('sex',axis=1,inplace=True)


#['INTERCEPT','age','fnlwgt','education-num',	'sex',	'capital-gain',	'capital-loss',	'hours-per-week',
# 'workclass_Without-pay',	'education_Doctorate', 'marital-status_Married-AF-spouse',	relationship_Not-in-family']

pre_rows,pre_cols = adult_test_16.shape
print("After dropping all the features we don't want. Here is the Adult wage training data set.\n"
        "It now has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")
adult_test_16.head()


This is the Adult wage training data set. It has 13788 instances, and it has 35 features.


After dropping all the features we don't want. Here is the Adult wage training data set.
It now has 13788 instances, and it has 17 features.




,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Government,workclass_Self-emp-not-inc,workclass_Without-pay,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Other-service,occupation_Priv-house-serv,race_Asian-Pac-Islander,wage
0,25,7,0,0,40,0,0,0,0,0,0,0,0,0,0,0,0
1,38,9,0,0,50,0,0,0,0,1,0,1,0,0,0,0,0
2,28,12,0,0,40,1,0,0,0,1,0,0,0,0,0,0,1
3,44,10,7688,0,40,0,0,0,0,1,0,0,0,0,0,0,1
5,34,6,0,0,30,0,0,0,0,0,0,0,0,1,0,0,0


### Part 3 Classification Task

Now we need to do the classification task on the dataset I altered.



#### a) classfication Task with abs value of coefficient > 0.6 --> 16 features
First, we use the KNN to see how it performs on the data. For this particular task, we could see that from the part 1 illustrations, there are a lot of linear pattern shown in the images. Hence, it would be better to use the Linear classifier. But we will try KNN here and compare it to the linear classifier later to see how it performs.

In [20]:
from sklearn.dummy import DummyClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

def getMean(inputList):
  # return: the average (the mean) for any list of values(numbers).
  
  return np.mean(inputList)

# slice the data

adult_train_06 = adult_train_06.values

X_train = adult_train_06[:,:-1]
y_train = adult_train_06[:,len(adult_train_06[0])-1]

adult_Test_16 = adult_test_16.values
X_test = adult_Test_16[:,:-1]
y_test = adult_Test_16[:,len(adult_Test_16[0])-1]

# KNN classifier
knnClass = KNeighborsClassifier(n_neighbors=3)
KNNClass = knnClass.fit(X_train, y_train)
knnClass_pred = KNNClass.predict(X_test)
knnScore = accuracy_score(y_test, knnClass_pred) * 100

knnClassScore = cross_val_score(KNNClass, X_test, y_test, cv=5)
knnClassEval = []
for i in knnClassScore:
  knnClassEval.append(i*100)

# ZeroRC
zr_clf = DummyClassifier(strategy="most_frequent")
zeroR = zr_clf.fit(X_train, y_train)
zeroR_pred = zeroR.predict(X_test)
zrScore = accuracy_score(y_test, zeroR_pred) * 100

print('KNN accuracy sklearn: {:.2f}%'.format(knnScore))
print('KNN accuracy sklearn: {:.2f}% ({:.2f}% / {:.2f}%)'.format(getMean(knnClassEval), min(knnClassEval), max(knnClassEval)))
print('ZeroR accuracy sklearn: {:.2f}%'.format(zrScore))

KNN accuracy sklearn: 82.92%
KNN accuracy sklearn: 82.79% (82.49% / 83.28%)
ZeroR accuracy sklearn: 74.96%


Here, we can see that KNN performs better than the baseline. KNN classifier is 82.92% which is 7.96% better than the baseline (74.96). This means that there might be also some culster(non-linear) patterns between the features.

Now we are going to use the Logistic Regression and the Naive Bayes to see how it performs. And we will also compare them to the KNN classifier. 

In [21]:

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# logistic Regression
lr_clf = LogisticRegression()
lr = lr_clf.fit(X_train,y_train)
lr_predY = lr.predict(X_test)
lr_score = accuracy_score(y_test, lr_predY)*100

#naive bayes
GNB_clf = GaussianNB()
GNB = GNB_clf.fit(X_train, y_train)
GNB_pred = GNB.predict(X_test)
gnb_score = accuracy_score(y_test, GNB_pred) *100


print('KNN accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(knnScore))
print()
print('ZeroR accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(zrScore))
print()
print('Logistic Regression accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(lr_score))
print()
print('Naive Bayes accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(gnb_score))
print()
print()



KNN accuracy with absolute value of coefficient of 0.8: 82.92%

ZeroR accuracy with absolute value of coefficient of 0.8: 74.96%

Logistic Regression accuracy with absolute value of coefficient of 0.8: 82.71%

Naive Bayes accuracy with absolute value of coefficient of 0.8: 80.02%




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


According to the accuracy we have, the accuracy of Logistic Regression and Naive Bayes are all better than the baseline. However, compare to the KNN classifier, KNN performs the best.  Logistic Regression performs better than the Naive Bayes and Naive Bayes performs the worst. 

This shows that there there might be more of a non-linear relationship between the 16 features and the output.

Based on the performance of the Naive Bayes, we can say that there might be a lot of overlaps between features distributions. This also make sense based on the illustrations above in part 1.

According to the listed 16 features we choose above in part 2, we have that capital-gain, education-num, hours-per-week and capital-loss have more influence to the output.

#### b) classification task with abs value of coefficient > 0.8 --> 13 features

In [22]:
print(sorted(coefficient))
selected_features = featureSelection(coefficient, 0.8, fetures_name)

print()
print("The choosen features with the absolute coefficient value that is bigger than 0.6 are:\n\n")
for feature in selected_features:
  data = '%.3f'%feature[0]
  print('{:^40}'.format(feature[1]), ":", '%10s'%data)



[-0.9648747434181131, -0.95495116665439, -0.868737198214596, -0.8417906610793756, -0.821871948002817, -0.8175022191330252, -0.6515291990830837, -0.5791625427380935, -0.41999389040525575, -0.3505509869507924, -0.19339580111724453, -0.18966969946816825, -0.11578351675410214, -0.08785462814358844, -0.06260818849557342, -0.035522832603041885, -0.028300917766683534, -0.010920196454361304, 0.16270000090547762, 0.19979550885858469, 0.2383883940733383, 0.3644507279244387, 0.5157621951672104, 0.5277151309807555, 0.6237940476880754, 0.6496318375921678, 0.8359876124124719, 1.6644956202342782, 1.7940640174395674, 1.9325829035477164, 2.38390141385391, 2.6914657055919564, 4.197511405040842, 11.082527536705479]

The choosen features with the absolute coefficient value that is bigger than 0.6 are:


       occupation_Farming-fishing        :     -0.965
        occupation_Other-service         :     -0.955
       occupation_Priv-house-serv        :     -0.869
          workclass_Government           : 

In [23]:
# leave the 13 features we chose for training set.
adult_train_13 = adult_train_copy.copy()
pre_rows,pre_cols = adult_train_13.shape
print("This is the Adult wage training data set. It has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")

adult_train_13.drop('race_Other',axis=1,inplace=True)
adult_train_13.drop('race_White',axis=1,inplace=True)
adult_train_13.drop('race_Black',axis=1,inplace=True)
adult_train_13.drop('race_Asian-Pac-Islander',axis=1,inplace=True)

adult_train_13.drop('occupation_Transport-moving',axis=1,inplace=True)
adult_train_13.drop('occupation_Tech-support',axis=1,inplace=True)
adult_train_13.drop('occupation_Sales',axis=1,inplace=True)
adult_train_13.drop('occupation_Prof-specialty',axis=1,inplace=True)
adult_train_13.drop('occupation_Machine-op-inspct',axis=1,inplace=True)
adult_train_13.drop('occupation_Handlers-cleaners',axis=1,inplace=True)
adult_train_13.drop('occupation_Craft-repair',axis=1,inplace=True)
adult_train_13.drop('occupation_Armed-Forces',axis=1,inplace=True)

adult_train_13.drop('marital-status_Separated',axis=1,inplace=True)
adult_train_13.drop('marital-status_Never-married',axis=1,inplace=True)
adult_train_13.drop('marital-status_Married-spouse-absent',axis=1,inplace=True)
adult_train_13.drop('marital-status_Widowed',axis=1,inplace=True)

adult_train_13.drop('workclass_Government',axis=1,inplace=True)
adult_train_13.drop('workclass_Without-pay',axis=1,inplace=True)
adult_train_13.drop('workclass_Self-emp-inc',axis=1,inplace=True)
adult_train_13.drop('workclass_Private',axis=1,inplace=True)
adult_train_13.drop('sex',axis=1,inplace=True)


#['INTERCEPT','age','fnlwgt','education-num',	'sex',	'capital-gain',	'capital-loss',	'hours-per-week',
# 'workclass_Without-pay',	'education_Doctorate', 'marital-status_Married-AF-spouse',	relationship_Not-in-family']

pre_rows,pre_cols = adult_train_13.shape
print("After dropping all the features we don't want. Here is the Adult wage training data set.\n"
        "It now has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")
adult_train_13.head()

# leave the 13 features we chose for testing set.
adult_test_13 = adult_test_copy.copy()
pre_rows,pre_cols = adult_test_13.shape
print("This is the Adult wage training data set. It has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")

adult_test_13.drop('race_Other',axis=1,inplace=True)
adult_test_13.drop('race_White',axis=1,inplace=True)
adult_test_13.drop('race_Black',axis=1,inplace=True)
adult_test_13.drop('race_Asian-Pac-Islander',axis=1,inplace=True)

adult_test_13.drop('occupation_Transport-moving',axis=1,inplace=True)
adult_test_13.drop('occupation_Tech-support',axis=1,inplace=True)
adult_test_13.drop('occupation_Sales',axis=1,inplace=True)
adult_test_13.drop('occupation_Prof-specialty',axis=1,inplace=True)
adult_test_13.drop('occupation_Machine-op-inspct',axis=1,inplace=True)
adult_test_13.drop('occupation_Handlers-cleaners',axis=1,inplace=True)
adult_test_13.drop('occupation_Craft-repair',axis=1,inplace=True)
adult_test_13.drop('occupation_Armed-Forces',axis=1,inplace=True)

adult_test_13.drop('marital-status_Separated',axis=1,inplace=True)
adult_test_13.drop('marital-status_Never-married',axis=1,inplace=True)
adult_test_13.drop('marital-status_Married-spouse-absent',axis=1,inplace=True)
adult_test_13.drop('marital-status_Widowed',axis=1,inplace=True)

adult_test_13.drop('workclass_Government',axis=1,inplace=True)
adult_test_13.drop('workclass_Without-pay',axis=1,inplace=True)
adult_test_13.drop('workclass_Self-emp-inc',axis=1,inplace=True)
adult_test_13.drop('workclass_Private',axis=1,inplace=True)
adult_test_13.drop('sex',axis=1,inplace=True)


#['INTERCEPT','age','fnlwgt','education-num',	'sex',	'capital-gain',	'capital-loss',	'hours-per-week',
# 'workclass_Without-pay',	'education_Doctorate', 'marital-status_Married-AF-spouse',	relationship_Not-in-family']

pre_rows,pre_cols = adult_train_13.shape
print("After dropping all the features we don't want. Here is the Adult wage testing data set.\n"
        "It now has", pre_rows, "instances, and it has", pre_cols, "features.\n\n")
adult_train_13.head()





This is the Adult wage training data set. It has 27504 instances, and it has 35 features.


After dropping all the features we don't want. Here is the Adult wage training data set.
It now has 27504 instances, and it has 14 features.


This is the Adult wage training data set. It has 13788 instances, and it has 35 features.


After dropping all the features we don't want. Here is the Adult wage testing data set.
It now has 27504 instances, and it has 14 features.




,age,education-num,capital-gain,capital-loss,hours-per-week,workclass_Self-emp-not-inc,marital-status_Married-AF-spouse,marital-status_Married-civ-spouse,occupation_Exec-managerial,occupation_Farming-fishing,occupation_Other-service,occupation_Priv-house-serv,occupation_Protective-serv,wage
0,39,13,2174,0,40,0,0,0,0,0,0,0,0,0
1,50,13,0,0,13,1,0,1,1,0,0,0,0,0
2,38,9,0,0,40,0,0,0,0,0,0,0,0,0
3,53,7,0,0,40,0,0,1,0,0,0,0,0,0
5,37,14,0,0,40,0,0,1,1,0,0,0,0,0


In [24]:
# slice the data

adult_Train_13 = adult_train_13.values

X_train = adult_Train_13[:,:-1]
y_train = adult_Train_13[:,len(adult_Train_13[0])-1]

adult_Test_13 = adult_test_13.values
X_test = adult_Test_13[:,:-1]
y_test = adult_Test_13[:,len(adult_Test_13[0])-1]

# KNN classifier
knnClass = KNeighborsClassifier(n_neighbors=3)
KNNClass = knnClass.fit(X_train, y_train)
knnClass_pred = KNNClass.predict(X_test)
knnScore = accuracy_score(y_test, knnClass_pred) * 100

knnClassScore = cross_val_score(KNNClass, X_test, y_test, cv=5)
knnClassEval = []
for i in knnClassScore:
  knnClassEval.append(i*100)

# ZeroRC
zr_clf = DummyClassifier(strategy="most_frequent")
zeroR = zr_clf.fit(X_train, y_train)
zeroR_pred = zeroR.predict(X_test)
zrScore = accuracy_score(y_test, zeroR_pred) * 100

# logistic Regression
lr_clf = LogisticRegression()
lr = lr_clf.fit(X_train,y_train)
lr_predY = lr.predict(X_test)
lr_score = accuracy_score(y_test, lr_predY)*100

#naive bayes
GNB_clf = GaussianNB()
GNB = GNB_clf.fit(X_train, y_train)
GNB_pred = GNB.predict(X_test)
gnb_score = accuracy_score(y_test, GNB_pred) *100


print('KNN accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(knnScore))
print()
print('ZeroR accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(zrScore))
print()
print('Logistic Regression accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(lr_score))
print()
print('Naive Bayes accuracy with absolute value of coefficient of 0.8: {:.2f}%'.format(gnb_score))
print()
print()

KNN accuracy with absolute value of coefficient of 0.8: 82.97%

ZeroR accuracy with absolute value of coefficient of 0.8: 74.96%

Logistic Regression accuracy with absolute value of coefficient of 0.8: 79.67%

Naive Bayes accuracy with absolute value of coefficient of 0.8: 79.82%




/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
